In [1]:
print("Hi")

Hi


In [2]:
%pwd

'e:\\Medico_chatbot\\Medico_Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'e:\\Medico_chatbot\\Medico_Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract Data from the PDF File

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data='Data/')

In [9]:
# Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of the chunks", len(text_chunks))

Length of the chunks 5860


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [24]:
from sentence_transformers import SentenceTransformer
print("sentence-transformers is working!")


sentence-transformers is working!


In [25]:
import sentence_transformers
embeddings = download_hugging_face_embeddings()

In [26]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [50]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [39]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "c:\Users\kdeepak_new\anaconda3\envs\medico\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-bsaqknd.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [52]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [41]:
# Embed each chunk and upsert the embeddings into yours pincone index.
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "c:\Users\kdeepak_new\anaconda3\envs\medico\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [42]:
# Load Exesting index 

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "c:\Users\kdeepak_new\anaconda3\envs\medico\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [43]:
docsearch

In [55]:
# This line of code give me three relevant answer

retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [56]:
retriver_docs = retriver.invoke("What is Acne?")
retriver_docs

[Document(id='2bb09099-2442-43d6-8931-cbc686a7d0a5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='42bcb28c-7257-4380-9bc3-5f49bf68de62', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [57]:
from langchain_google_genai import GoogleGenerativeAI
import os

# Initialize the LLM
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.4, max_output_tokens=500)

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI
import os

# Initialize the Gemini-Pro LLM
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.4, max_output_tokens=500)

# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved context to answer the questions. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



In [62]:
# Step 4: Create the Question-Answering Chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Step 5: Create the Retrieval-Augmented Generation (RAG) Chain
rag_chain = create_retrieval_chain(retriver, question_answer_chain)


In [63]:
response = rag_chain.invoke({"input": "What is Acne?"})


In [64]:
response

{'input': 'What is Acne?',
 'context': [Document(id='2bb09099-2442-43d6-8931-cbc686a7d0a5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
  Document(id='42bcb28c-7257-4380-9bc3-5f49bf68de62', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers

In [65]:
print(response["answer"])  # ✅ Correct key

Acne is a skin disorder in which the sebaceous glands become inflamed. It is characterized by pimples on the face, chest, and back. Acne vulgaris is the most common type of acne, affecting nearly 17 million people in the United States.


In [70]:
response = rag_chain.invoke({"input": "what is Achondroplasia?"})
print(response["answer"])

Achondroplasia is a condition in which the development of cartilage and bone is disturbed. It is usually diagnosed at birth due to the characteristic appearance of the newborn. The disorder appears in approximately one in every 10,000 births.
